<a href="https://colab.research.google.com/github/mouadnechchad/FaceNet-for-Face-Recognition/blob/main/Face_reco_FaceNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Face Recognition using Transfer Learning from FaceNet**

In [ ]:
!pip install keras-facenet

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 26.9 MB/s eta 0:00:00
  Created wheel for keras-facenet: filename=keras_facenet-0.3.2-py3-none-any.whl size=10370 sha256=d7e6ec1e4c298f4f6b21456344e5f8652aaeb8144aa1dd71e563045c924731a5
  Stored in directory: /root/.cache/pip/wheels/1d/d8/a9/85cf04ea29321d2afcb82c0caaafdca9195385f9d68cbc7185
Successfully built keras-facenet


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from keras_facenet import FaceNet
embedder = FaceNet()

In [ ]:
import matplotlib.image as mpimg
import cv2
import numpy as np
import pandas as pd

## Loadng dataset


In [ ]:
import cv2
import numpy as np
import pandas as pd

def load_train_data(n):
    lst = []
    for i in range(1, n + 1):
        x = cv2.imread(f'/content/drive/MyDrive/Computer_Vision/Face_Recogniton/Dataset/Train/{i}.jpg')
        x = cv2.resize(x, (160, 160))  # Resize images to a consistent shape (160, 160, 3 for RGB images)
        lst.append(x)
    df = np.array(lst)
    y_train = pd.read_csv('/content/drive/MyDrive/Computer_Vision/Face_Recogniton/Dataset/Train/y_train.csv', header=None)
    y_train = y_train.values

    return df, y_train

In [ ]:
import cv2
import numpy as np
import pandas as pd

def load_test_data(n):
    lst = []
    for i in range(1, n + 1):
        x = cv2.imread(f'/content/drive/MyDrive/Computer_Vision/Face_Recogniton/Dataset/Test/{i}.jpg')
        x = cv2.resize(x, (160, 160))  # Resize images to a consistent shape (160, 160, 3 for RGB images)
        lst.append(x)
    df = np.array(lst)
    y_test = pd.read_csv('/content/drive/MyDrive/Computer_Vision/Face_Recogniton/Dataset/Test/y_test.csv', header=None)
    y_test = y_test.values

    return df, y_test

In [ ]:
'''
Loading the data set
'''
X_train_orig,y_train = load_train_data(12)
X_test_orig,y_test = load_test_data(8)
print('X_Train data shape=',X_train_orig.shape)
print('X_Test data shape=',X_test_orig.shape)
print('y_Train data shape=',y_train.shape)
print('y_Test data shape=',y_test.shape)

X_Train data shape= (12, 160, 160, 3)
X_Test data shape= (8, 160, 160, 3)
y_Train data shape= (12, 1)
y_Test data shape= (8, 1)


In [ ]:
X_train = embedder.embeddings(X_train_orig)
X_test = embedder.embeddings(X_test_orig)
print('Train embed shape=',X_train.shape)
print('Test embed shape=',X_test.shape)

1/1 [==============================] - 1s 845ms/step
Train embed shape= (12, 512)
Test embed shape= (8, 512)


## Train the model

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
model = LogisticRegression()

In [14]:
model.fit(X_train,y_train.reshape(-1));

In [15]:
#Predicted faces
y_pred = model.predict(X_test).reshape(-1,1)
y_pred

array([[1],
       [1],
       [2],
       [2],
       [3],
       [3],
       [4],
       [4]])

In [16]:
#True faces
y_test

array([[1],
       [1],
       [2],
       [2],
       [3],
       [3],
       [4],
       [4]])

## Model Performance

In [17]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 1.0


- We obtaine a perfect model performance.

## Make Predictions

In [18]:
import cv2
import numpy as np

def predict_label_for_image(model, image_path, embedder):
    """
    Make predictions using a trained logistic regression model on an input image.

    Args:
    - model: The trained logistic regression model.
    - image_path: Path to the input image.
    - embedder: The function that generates embeddings from images.

    Returns:
    - predicted_label: The predicted label for the input image.
    """
    # Load and preprocess the image
    img = cv2.imread(image_path)
    img = cv2.resize(img, (160, 160))  # Resize to the same shape as your training data
    embedding = embedder.embeddings(np.array([img]))

    # Use the logistic regression model to make a prediction
    predicted_label = model.predict(embedding)

    return predicted_label


In [20]:
predicted_label = predict_label_for_image(model, '/content/drive/MyDrive/Computer_Vision/Face_Recogniton/Dataset/Test/2.jpg', embedder)
predicted_label

1/1 [==============================] - 0s 91ms/step


array([1])

- We get the predicted label that corresponds to a person.


In [ ]:
y_test[0]

array([1])

## Save the model

In [ ]:
!pip install joblib

In [ ]:
import joblib

# Assuming you have a trained logistic regression model named 'model'
# Save the model to a file
model_filename = '/content/drive/MyDrive/Computer_Vision/Face_Recogniton/face_recognition.pkl'
joblib.dump(model, model_filename)


['/content/drive/MyDrive/Computer_Vision/Face_Recogniton/face_recognition.pkl']